In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Reading the files

population_proj = pd.read_csv("../data/raw/summary-statistics.csv", skiprows=3)

In [7]:
population_proj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            73 non-null     object 
 1   Total male population (000s)    71 non-null     object 
 2   Total female population (000s)  71 non-null     object 
 3   Total population (000s)         77 non-null     object 
 4   Median age                      71 non-null     float64
 5   Sex ratio                       71 non-null     float64
 6   Median age at death             71 non-null     object 
 7   Total fertility                 71 non-null     object 
 8   Child dependency                71 non-null     float64
 9   Elderly dependency              71 non-null     float64
 10  Total dependency                71 non-null     float64
dtypes: float64(5), object(6)
memory usage: 7.1+ KB
